# Intensity Prediction 

This notebook is prepared to be run in Google [Colaboratory](https://colab.research.google.com/). In order to train the model faster, please change the runtime of Colab to use Hardware Accelerator, either GPU or TPU.

This notebook presents a short walkthrough the process of reading a dataset and training a model for intensity prediction. The dataset is an example dataset extracted from a ProteomeTools dataset generated in the **Chair of Bioanalytics** at the **Technical University of Munich**.

The framework being used is a custom wrapper on top of Keras/TensorFlow. The working name of the package is for now DLOmix -  `dlomix`.

In [ ]:
# install the DLOmix package in the current environment using pip

!python -m pip install -q dlomix==0.0.4

In [ ]:
!python -m pip install wandb

The available modules in the framework are as follows:

In [ ]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
print([x for x in dir(dlomix) if not x.startswith("_")])

- `constants`: constants to be used in the framework (e.g. Aminoacid alphabet mapping)
- `data`:  classes for representing dataset, wrappers around TensorFlow Dataset
- `eval`: custom evaluation metrics implemented in Keras/TF to work as `metrics` for model training
- `layers`: custom layer implementation required for the different models
- `models`: different model implementations for Retention Time Prediction
- `pipelines`: complete pipelines to run a task (e.g. Retention Time prediction)
- `utils`: helper modules

**Note**: reports and pipelines are work-in-progress, some funtionalities are not complete.

## 0. Import and Initialize Weights and Biases


In [ ]:
# import wandb and the Keras Callback

import wandb
from wandb.keras import WandbCallback

In [ ]:
# enter project name
project_name = 'dlomix_intensity'

## 1. Load Data

We can import the dataset class and create an object of type `RetentionTimeDataset`. This object wraps around TensorFlow dataset objects for training+validation or for testing. This can be specified by the arguments `val_ratio` and `test`.

In [ ]:
from dlomix.data import IntensityDataset

In [ ]:
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/proteomeTools_train_val.csv'
BATCH_SIZE = 64

int_data = IntensityDataset(data_source=TRAIN_DATAPATH,
                              seq_length=30, batch_size=BATCH_SIZE,collision_energy_col='collision_energy', val_ratio=0.2, test=False)

Now we have an RT dataset that can be used directly with standard or custom `Keras` models. This wrapper contains the splits we chose when creating it. In our case, they are training and validation splits. To get the TF Dataset, we call the attributes `.train_data` and `.val_data`. The length is now in batches (i.e. `total examples = batch_size x len`)

In [ ]:
 "Training examples", BATCH_SIZE * len(int_data.train_data)

In [ ]:
"Validation examples", BATCH_SIZE * len(int_data.val_data)

## 2. Model

We can now create the model. We will use a simple Prediction with a conv1D encoder. It has the default working arguments, but most of the parameters can be customized.

**Note**: Important is to ensure that the padding length used for the dataset object is equal to the sequence length passed to the model.

In [ ]:
from dlomix.models import PrositIntensityPredictor

In [ ]:
model = PrositIntensityPredictor(seq_length=30)

## 3. Training

We can then train the model like a standard Keras model. The training parameters here are from Prosit, but other optimizer parameters can be used.  

In [ ]:
#imports

import tensorflow as tf
from dlomix.eval import TimeDeltaMetric
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
tf.get_logger().setLevel('ERROR')

In [ ]:
# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom timedelta metric
model.compile(optimizer=optimizer,
              loss=masked_spectral_distance,
              metrics=['mse',masked_pearson_correlation_distance])

We store the result of training so that we can explore the metrics and the losses later. We specify the number of epochs for training and pass the training and validation data as previously described.

In [ ]:
wandb.init(project=project_name, name='walktrough')
history = model.fit(int_data.train_data,
                    validation_data=int_data.val_data,
                    epochs=15,callbacks=[WandbCallback(save_model=False)])

In [ ]:
# Mark the run as finished
wandb.finish()

## 3. Testing and Reporting

We can create a test dataset to test our model. Additionally, we can use the reporting module to produce plots and evaluate the model.

Note: the reporting module is still in progress and some functionalities might easily break.

In [ ]:
# create the dataset object for test data

TEST_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/proteomeTools_test.csv'

test_int_data = IntensityDataset(data_source=TEST_DATAPATH,
                              seq_length=30, collision_energy_col='collision_energy',batch_size=32, test=True)

In [ ]:
# use model.predict from keras directly on the testdata

predictions = model.predict(test_int_data.test_data)

In [ ]:
from dlomix.reports import IntensityReport

# create a report object by passing the history object and plot different metrics
report = IntensityReport(output_path="./output", history=history)

In [ ]:
# you can generate a complete report for intensity by calling generate_report
# the function takes the test dataset object and the predictions as arguments

report.generate_report(test_int_data, predictions)

In [ ]:
# you can also manually see the results by calling other utility functions
from dlomix.reports.postprocessing import normalize_intensity_predictions


predictions_df = report.generate_intensity_results_df(test_int_data, predictions)
predictions_df.head()

In [ ]:
predictions_acc = normalize_intensity_predictions(predictions_df)
predictions_acc.head()

In [ ]:
predictions_acc['spectral_angle'].describe()

In [ ]:
import seaborn as sns

sns.violinplot(predictions_acc['spectral_angle'])